In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = "api-key")
client

In [ ]:
uploaded_file = client.files.create(
    file=open("story.txt",'rb'),
    purpose='assistants'
)

In [ ]:
uploaded_file

In [ ]:
# Create a vector store caled Story Database
vector_store = client.beta.vector_stores.create(name="Story Data")
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=[open("story.txt",'rb')]
)

In [ ]:
print(file_batch.status)
print(file_batch.file_counts)

In [ ]:
assistant = client.beta.assistants.create(
  name="Story helper",
  instructions="You are a motivator who answers the question based on the story file",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

In [ ]:
thread = client.beta.threads.create()
thread

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who is the hero of the story?"
)

In [ ]:
message

In [ ]:
assistant

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [ ]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [ ]:
run.status

In [ ]:
message

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages

In [ ]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status=="completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)
        break;